# Non-interactive Windows Shells

https://attack.mitre.org/techniques/T1059/

Search for `powershell` or `cmd` running non-interactively; i.e. not started by `explorer.exe`. 

This isn't necessarily malicious, so further environment-specific filtering woul dbe necessry to operationalize.

Dataset from https://securitydatasets.com/notebooks/atomic/windows/defense_evasion/SDWIN-201022025808.html



In [1]:
ps = GET process
     FROM https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/defense_evasion/host/cmd_mshta_vbscript_execute_psh.zip
     WHERE [process:binary_ref.name IN ('cmd.exe', 'powershell.exe') AND process:parent_ref.binary_ref.name != 'explorer.exe']
DISP ps ATTR pid, binary_ref.name, command_line, parent_ref.pid, parent_ref.binary_ref.name

pid 
 binary_ref.name 
 command_line 
 parent_ref.pid 
 parent_ref.binary_ref.name 
 
 
 
 
 9572 
 powershell.exe 
 None 
 
 None 
 
 
 9572 
 powershell.exe 
 "C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe" -noexit -command Get-Service sysmon 
 10980.0 
 mshta.exe 
 
 
 
 Block Executed in 3 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 artifact* 
 directory* 
 file* 
 process* 
 user-account* 
 windows-registry-key* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 ps 
 process 
 41 
 668 
 668 
 766 
 768 
 633 
 156 
 398 
 668 
 668 
 
 
 *Number of related records cached.

We got a hit (process ID 9572), so we can manually inspect the command line to figure out what's going on.  The `Get-Service` cmdlet returns information about services on the host.  Here's it's requesting information on `sysmon`, an important service for securty monitoring.  Perhaps this is some reconnaissance?

Let's try to see where this command came from by moving up the process tree...

In [2]:
parents = FIND process CREATED ps
DISP parents ATTR pid, name, command_line, parent_ref.pid, parent_ref.binary_ref.name

pid 
 name 
 command_line 
 parent_ref.pid 
 parent_ref.binary_ref.name 
 
 
 
 
 10980 
 mshta.exe 
 mshta vbscript:Execute("CreateObject(""Wscript.Shell"").Run ""powershell -noexit -command Get-Service sysmon"":close") 
 5888.0 
 cmd.exe 
 
 
 10980 
 mshta.exe 
 None 
 
 None 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 artifact* 
 directory* 
 file* 
 process* 
 user-account* 
 windows-registry-key* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 parents 
 process 
 11 
 145 
 809 
 913 
 1005 
 808 
 185 
 417 
 809 
 809 
 
 
 *Number of related records cached.

The parent process is `mshta.exe` - the Microsoft HTML Application Host.  This program can be used to run VBScript (as it's doing here), among other things.  It has been known to be used in "living off the land" attacks (see https://attack.mitre.org/techniques/T1218/005/ to read about this particular usage).

Can we take another step up the tree?

In [3]:
grandparents = FIND process CREATED parents
DISP grandparents ATTR pid, name, command_line, parent_ref.pid, parent_ref.binary_ref.name

pid 
 name 
 command_line 
 parent_ref.pid 
 parent_ref.binary_ref.name 
 
 
 
 
 5888 
 cmd.exe 
 None 
 None 
 None 
 
 
 5888 
 cmd.exe 
 "C:\windows\system32\cmd.exe" 
 None 
 None 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 artifact* 
 directory* 
 file* 
 process* 
 user-account* 
 windows-registry-key* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 grandparents 
 process 
 2 
 5 
 144 
 150 
 243 
 148 
 31 
 19 
 144 
 144 
 
 
 *Number of related records cached.

So `mshta.exe` was run by `cmd.exe`, but here we hit a dead end.  There's no further info in this data, and if we read the description of how this data was generated we see that `cmd.exe` was run manually in the adversarial simulation.